In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [1]:
import tushare as ts

ts.set_token('b44a79bfc0d0985920d0328c722c4b32c64a6f4ea73d6937469fcd8c')
pro = ts.pro_api()

In [7]:
import xlrd
import pandas as pd
from openpyxl import load_workbook

excel_name = r'/Users/lsrsusan/Desktop/code/python数据分析/dsa5205/DSA5205.xls'
# 写入文件位置，*处填入文件位置

Wb = xlrd.open_workbook('DSA5205.xls') 
# 获取工作表中所有的表格

Sheets = Wb.sheet_names()
# 获取工作表中所有表格名

Merge_data = pd.DataFrame()
# 建立空DataFrame，存储数据

for i in range(len(Sheets)):  
# 循环遍历所有Sheets
    
    df = pd.read_excel(excel_name, sheet_name=i)
    df1= df[list(df.columns)]
    df1['Quarter']=Sheets[i]
    Merge_data = Merge_data.append(df1)
Merge_data[Merge_data['index']=="AAPL"]

/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_data.append(df1)
/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_data.append(df1)
/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_data.append(df1)
/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_d

In [12]:
msft = yf.Ticker("YI AAPL")

# get stock info
msft.info
msft.quarterly_balancesheet

df = pro.us_tradecal(start_date='20210101', end_date='20220731')
df=df[df['is_open']==1]
df=df.reset_index(drop=True)
df['cal_date']

{'regularMarketPrice': None, 'preMarketPrice': None, 'logo_url': ''}

In [33]:
from datetime import datetime
import math

now = datetime.datetime(2015, 1, 1, 0, 0, 0, 0)
present_quarter = int(math.ceil(now.month/3.))
completed_quarter = (present_quarter - 1) if present_quarter != 1 else 4

df['cal_date']=[datetime.strptime(i,"%Y%m%d") for i in list(df['cal_date'].values)]
df['Quarter']=df['cal_date'].map(lambda x:str(x.year)+"Q"+str(math.ceil(x.month/3)))
df=df.iloc[:,[0,-1]]
df.rename(columns={'cal_date':'date'},inplace=True)
df1=pd.merge(df,Merge_data,on="Quarter",how='right')
df1=df1.set_index("date")
df1=df1.sort_index()
df1.reset_index(inplace=True)
df1=df1.set_index(['date', 'index'])

In [91]:
df1.to_csv("financial_quality_factor.csv")

In [43]:
financial_factor = pd.read_csv("../../Data/financial_quality_factor.csv")
stock_industry = pd.read_csv("../../Data/stock_industry.csv")
missings = pd.read_csv("../../Data/missing_stocks.csv")

In [44]:
financial_factor = financial_factor.loc[:, ["date","index","RETURN_ON_ASSET","RETURN_ON_CAP","RETURN_COM_EQY"]].rename(columns={"date":"Date","index":"Index","RETURN_ON_ASSET":"ROA","RETURN_ON_CAP":"ROC","RETURN_COM_EQY":"ROE"})
financial_factor = financial_factor.merge(stock_industry, on="Index")

In [46]:
stock_index = financial_factor.loc[:, "Index"].unique()
for index in stock_index:
    financial_factor.loc[financial_factor["Index"]==index, "ROA"] = financial_factor.loc[financial_factor["Index"]==index, "ROA"].replace(0, np.nan)
    financial_factor.loc[financial_factor["Index"]==index, "ROE"] = financial_factor.loc[financial_factor["Index"]==index, "ROE"].replace(0, np.nan)
    financial_factor.loc[financial_factor["Index"]==index, "ROC"] = financial_factor.loc[financial_factor["Index"]==index, "ROC"].replace(0, np.nan)
    financial_factor.loc[financial_factor["Index"]==index, "ROA"] = financial_factor.loc[financial_factor["Index"]==index, "ROA"].fillna(financial_factor.loc[financial_factor["Index"]==index, "ROA"].mean())
    financial_factor.loc[financial_factor["Index"]==index, "ROE"] = financial_factor.loc[financial_factor["Index"]==index, "ROE"].fillna(financial_factor.loc[financial_factor["Index"]==index, "ROE"].mean())
    financial_factor.loc[financial_factor["Index"]==index, "ROC"] = financial_factor.loc[financial_factor["Index"]==index, "ROC"].fillna(financial_factor.loc[financial_factor["Index"]==index, "ROC"].mean())

In [62]:
financial_factor = financial_factor.loc[~financial_factor["Index"].isin(missings["Index"]), :]
financial_factor = financial_factor.loc[financial_factor["ROE"].notna()]
financial_factor = financial_factor.loc[financial_factor["industry"].notna()]
financial_factor = financial_factor.loc[:, ["Date","Index","ROA","ROE"]]

In [69]:
financial_factor.to_csv("../../Factors/Financial-Quality_Factor.csv")